Install and import the following packages 

In [ ]:
import pandas as pd
import IPython 
import geopandas as gpd
import matplotlib
import requests
import bs4
from bs4 import BeautifulSoup
import json
import folium
import requests
from google_images_search import GoogleImagesSearch
from IPython.display import Image

In [1]:
#Install the better_ebirdAPI
#its not a package yet because i need figure that out so this doesnt work 
!pip install better_ebirdAPI


ERROR: Could not find a version that satisfies the requirement better_ebirdAPI (from versions: none)
ERROR: No matching distribution found for better_ebirdAPI


In [ ]:
# Define all necessary API and CX keys
# if you dont have access to google API that is okay, you just wont be able to use the functions coded with 'GAPI'
google_apikey = "paste your google api key here"
google_CX = "paste your google CX here"
ebird_api_key = "paste your ebird API key here"

In [ ]:
# read the following csv file. this is necessary for the following fuctions: com_to_sci, sci_to_com, list_types
species_data = pd.read_csv("PFW-species-translation-table.csv")
species_data